In [1]:
import spiketoolkit as st
import spikeextractors as se
import numpy as np
import spikemetrics.metrics as metrics
from spikemetrics.common import Epoch

17:29:51 [I] klustakwik KlustaKwik2 version 0.2.6
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/networkx-2.2-py3.7.egg/networkx/classes/reportviews.py:95: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable


In [2]:
# workshop_path = "/Users/colehurwitz/"
workshop_path = "/afs/inf.ed.ac.uk/user/s17/s1727843/spiketutorials/Spike_sorting_workshop_2019/"
recording = se.MEArecRecordingExtractor(recording_path= workshop_path + "recordings_36cells_four-tetrodes_30.0_10.0uV_20-06-2019_14_48.h5")


Could not load plane information. Assuming probe is in yz plane


In [3]:
recording = se.load_probe_file(recording, workshop_path + 'tetrode_16.prb')

In [4]:
st.sorters.KlustaSorter.default_params()

{'probe_file': None,
 'adjacency_radius': None,
 'threshold_strong_std_factor': 5,
 'threshold_weak_std_factor': 2,
 'detect_sign': -1,
 'extract_s_before': 16,
 'extract_s_after': 32,
 'n_features_per_channel': 3,
 'pca_n_waveforms_max': 10000,
 'num_starting_clusters': 50}

In [ ]:
sorting = st.sorters.run_klusta(recording, threshold_weak_std_factor=1)

Klusta units are not unique! Using unique unit ids


In [ ]:
num_spikes = 0
for unit_id in sorting.get_unit_ids():
    num_spikes += len(sorting.get_unit_spike_train(unit_id))
print(num_spikes)

5931


In [ ]:
st.postprocessing.export_to_phy(recording, sorting, output_folder='phy_results')

In [ ]:
metric_info = st.postprocessing.get_quality_metric_data(recording, sorting, nPC=3, ms_before=1., ms_after=2., dtype=None, max_num_waveforms=np.inf, 
                                                        max_num_pca_waveforms=np.inf, save_waveforms=False, verbose=False)

In [ ]:
spike_times = recording.frame_to_time(metric_info[0])
spike_clusters = metric_info[1].astype(int)
amplitudes = metric_info[2]
channel_map = metric_info[3]
pc_features = metric_info[4]
pc_feature_ind = metric_info[5]

In [ ]:
params = {
          "isi_threshold" : 0.0015,
          "min_isi" : 0.000166,
          "num_channels_to_compare" : 13,
          "max_spikes_for_unit" : 500,
          "max_spikes_for_nn" : 10000,
          "n_neighbors" : 4,
          'n_silhouette' : 10000,
          "quality_metrics_output_file" : "metrics.csv",
          "drift_metrics_interval_s" : 51,
          "drift_metrics_min_spikes_per_interval" : 10
         }

In [ ]:
epochs_dict = {
    'red': (300000, 960000),
    'blue': (0, 300000)
}

In [ ]:
epochs = []
for key in list(epochs_dict.keys()):
    epoch_name = key
    epoch_start_frame = epochs_dict[key][0]
    epoch_end_frame = epochs_dict[key][1]
    epochs.append(Epoch(name=key, start_time=recording.frame_to_time(epoch_start_frame), end_time=recording.frame_to_time(epoch_end_frame)))
epochs = sorted(epochs, key=lambda epoch: epoch.start_time)

In [ ]:
metrics_pd = metrics.calculate_metrics(spike_times=spike_times, spike_clusters=spike_clusters, amplitudes=amplitudes, channel_map=channel_map, pc_features=pc_features,\
                                       pc_feature_ind=pc_feature_ind, params=params, epochs = None)

Calculating isi violations
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating presence ratio
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating firing rate
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating amplitude cutoff
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒ 100% 
Calculating PC-based metrics
 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░ 36% 

/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/afs/inf.ed.ac.uk/user/s17/s1727843/miniconda3/envs/spikeinterface/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


 ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒░░░░░░░░░░░░░ 68% 

In [ ]:
metrics_pd